In [1]:
import pandas as pd
import re
import seaborn as sns
import glob
import cloudpickle
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df_reports = pd.read_csv("diseasedata/wiki_list.csv",index_col=None, header=0)
df_reports=df_reports[['name','ref','views']]
df_reports.columns=['Disease','Ref','Count']
df_reports=df_reports.dropna()

In [3]:
import cloudpickle
disease_lookup = cloudpickle.load(open("disease_lookup.pkl", "rb" ) )
disease_list = cloudpickle.load(open("disease_list_stats.pkl", "rb" ) )

In [4]:
for dis in disease_list:
    dis['wiki_ref_freq']=0

In [5]:
from nltk import ngrams

def get_toks(txt,ngram_length):
    return [item for sublist in \
             [[" ".join(grams) for grams in ngrams(txt.split(), n)] \
             for n in range(ngram_length)] \
             for item in sublist]

def get_dislist_simple(diseases,text):
    choptext=dict([[a,0] for a in get_toks(text,4)])
    return [k for k in diseases if k in choptext]

for i,repo in enumerate(df_reports['Disease'].values):
    if(i%1000==0):
        print i
    txt = df_reports.loc[df_reports['Disease']==repo,'Disease'].values[0].lower()
    cnt = df_reports.loc[df_reports['Disease']==repo,'Count'].values[0]
    wiki= df_reports.loc[df_reports['Disease']==repo,'Ref'].values[0]
    alldiags = get_dislist_simple(disease_lookup.keys(),txt)
    for k in set([disease_lookup[x] for x in alldiags]):
        disease_list[k]['wiki_freq']=disease_list[k]['wiki_freq']+cnt
        if(disease_list[k]['wiki_ref_freq']<cnt):
            disease_list[k]['wiki_ref_freq']=cnt
            disease_list[k]['wiki_ref']=wiki

0
1000
2000
3000
4000


In [6]:
with open("disease_list_stats.pkl", 'wb') as handle:
    cloudpickle.dump(disease_list, handle)

In [ ]:
df_reports['Disease']